In [1]:
import pickle
from scipy.spatial.distance import cosine

sasho = pickle.load(open('sasho_bert_vectors_for_cosine.pickle', 'rb'))
utah = pickle.load(open('utah_bert_vectors_for_cosine.pickle', 'rb'))
keys_sasho = [key[0] for key in sasho]
keys_utah = [key[0] for key in utah]
intersection = list(set(keys_sasho) & set(keys_utah))

In [7]:
sasho_vectors_intersecting = []
utah_vectors_intersecting = []
utah = dict(utah)
sasho = dict(sasho)
print(utah["[CLS]"])

for intersecting_word in intersection:
    sasho_vectors_intersecting.append(tuple((intersecting_word, sasho[intersecting_word])))
    utah_vectors_intersecting.append(tuple((intersecting_word, utah[intersecting_word])))

cosine_distances_with_scipy = []
for sa, ut in zip (sasho_vectors_intersecting, utah_vectors_intersecting):
    assert sa[0] == ut[0], "strings from intersection should be the same, something is wrong!"
    cosine_distances_with_scipy.append(tuple((sa[0], cosine(sa[1], ut[1]))))

[-0.43093973  0.02270981  0.18791805 ... -0.5942187  -0.11230399
  0.5877969 ]


In [12]:
cos_scipy_sorted = sorted(cosine_distances_with_scipy, key=lambda tup: tup[1])
for i, val in enumerate(cos_scipy_sorted):
    if i == 0: print("Top 10") 
    if i == len(cos_scipy_sorted)-10: print("\nBottom 10")
    if 0 < i < 10 or len(cos_scipy_sorted)-10 < i < len(cos_scipy_sorted):
        print("{} {}".format(val[0], val[1]))
    

Top 10
terminating 0.0
life 0.0
##cycle 0.0
remove 0.0009971261024475098
young 0.001008152961730957
too 0.0010106563568115234
status 0.012241601943969727
nova 0.013717353343963623
##eti 0.01682746410369873

Bottom 10
on 0.4524269700050354
instance 0.45709383487701416
the 0.45835477113723755
" 0.4652404189109802
##d 0.49717140197753906
##m 0.5103933215141296
for 0.5324618518352509
t 0.6007658839225769
' 0.9519659355282784


['thrown', 'len', 'to', 'its', 'dd', '!', '##hr', 'this', '_', '##lo', 'swap', 'sync', 'default', 'error', 'id', 'local', 'compute', 'node', 'instance', 'from', 'power', 'specified', '/', 'li', 't', '##type', '##b', 'the', 'host', '##list', '##eti', 'not', 'in', '<', 'database', 'count', '##v', 'disk', 'memory', 'e', 'manager', '-', 'build', 'remove', 'at', 'file', 'status', 'has', '##ys', 'valid', 'time', '"', 'domain', 'terminating', 'image', 'of', 'complete', 'is', 'server', 'updated', 'pending', 'mb', 're', 'during', 'base', 'f', 'any', 'event', 'took', 'running', 'd', '(', '##cate', 'or', 'nova', ':', 'instances', 'run', '[SEP]', 'c', 'token', 'da', 'state', 'ph', '##ed', 'b', 'did', 'network', 'del', 'deal', 'supported', '>', '##m', 'exception', '##on', 'young', '##zing', 'unable', 'v', 'life', 'on', 'var', 'created', 'fe', '##oni', 'gb', 'failed', "'", '##cycle', 'available', ')', '##d', 'attempting', '##i', '##ing', 'service', 'http', '.', 'a', 'fetch', 'while', 'match', 'claim